In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

def actual_distribution():
    x=np.arange(0,1,0.05);
    y=np.sin(np.power(x,2)+1)
    return x,y

def add_noise(y_curve):
    mu=0
    sigma=0.03
    noise=np.random.normal(mu,sigma,len(y_curve))
    y_noise=y_curve+noise
    return y_noise

def numpy_poly_fitting(x,y,M):
    z=np.polyfit(x,y,M)
    f=np.poly1d(z)
    return f

def Gaussian_basis_Func(x,mu,s):
    G = np.exp(-np.power((x-mu),2)/(2*s*s))
    return G

def full_Bayesian_fitting(x,y,M,alpha,beta,s):
    N=len(x)
    print('N =',N)
    print('M =',M)
    mx=np.zeros([N,M+1])
    vy=np.zeros([N,1])
    I=np.identity(M+1)
    Ialpha=alpha*I
    
    for n in range(0,N):
        for m in range(0,M+1):
            if m == 0:
                mx[n][m]=1
            else:
                mu = m/(M+1)
                mx[n][m]=Gaussian_basis_Func(x[n],mu,s)
        vy[n][0]=y[n]
    
    mxx = np.dot(mx.T, mx)
    imxx = linalg.inv (beta * mxx + Ialpha)# covariance matrix 
    tmp=np.dot(imxx,mx.T)
    w=np.dot(beta*tmp,vy)    
    return w

def full_Bayesian_plot(x,w,s):
    M=len(w)-1
    N=len(x)
    
    mx=np.zeros([N,M+1])
    
    for n in range(0,N):
        for m in range(0,M+1):
            if m == 0:
                mx[n][m]=1
            else:
                mu = m/(M+1)
                mx[n][m]=Gaussian_basis_Func(x[n],mu,s)
   
    y=np.dot(mx,w)
    return x,y

# def predictive_distribution(x, point_num):
#     SD = [] #standard deviation
#     Phi = np.zeros([len(x_noise), M+1])
#     I=np.identity(M+1)
#     Ialpha=alpha*I
    
#     for n in range(0,len(x_noise)):
#         for m in range(0,M+1):
#             if m == 0:
#                 Phi[n][m] = 1
#             else:
#                 mu = m / (M+1)
#                 Phi[n][m] = Gaussian_basis_Func(x[n],mu,s)
#     #print('point_num =',point_num)
#     phi = np.zeros([point_num, M+1])
#     for n in range(0, point_num):
#         for m in range(0, M+1):
#             mu = m / (M+1)
#             phi[n][m] = Gaussian_basis_Func(x[n],mu,s)
           
#     SN = linalg.inv(alpha * Ialpha + beta * np.dot(phi.T, phi))
#     #print('SN =',SN)
#     S_new = np.zeros(50)
#     for i in range(0,len(x_noise)):
#         S_new[i] = 1/beta + np.dot(np.dot(Phi[i],SN),Phi[i].T)
#         #SD.append(np.sqrt(S_new))
#     return S_new

def predictive_distribution(x,y,M,alpha,beta,s):
    N=len(x)
    print('N =',N)
    print('M =',M)
    mx=np.zeros([N,M+1])
    vy=np.zeros([N,1])
    I=np.identity(M+1)
    Ialpha=alpha*I
    
    for n in range(0,N):
        for m in range(0,M+1):
            if m == 0:
                mx[n][m]=1
            else:
                mu = m/(M+1)
                mx[n][m]=Gaussian_basis_Func(x[n],mu,s)
        vy[n][0]=y[n]
    
    mxx = np.dot(mx.T, mx)
    imxx = linalg.inv (beta * mxx + Ialpha)# covariance matrix 
    tmp=np.dot(imxx,mx.T)
    mn=np.dot(beta*tmp,vy)    
    return imxx,mn

def predictive_distribution_plot(x,mn,SN,s,beta):
    M=len(w)-1
    N=len(x)
    
    mx=np.zeros([N,M+1])
    
    for n in range(0,N):
        for m in range(0,M+1):
            if m == 0:
                mx[n][m]=1
            else:
                mu = m/(M+1)
                mx[n][m]=Gaussian_basis_Func(x[n],mu,s)
                
    M_star = np.zeros([50,1])
    S_star = np.zeros(50)
    for i in range(N):
        M_star[i] = np.dot(mx[i], mn)
        S_star[i] = 1/beta + np.dot(np.dot(mx[i],SN),mx[i].T)
    return S_star, M_star 
#===============================================
print('start...')

M=8
alpha=0.5
beta=10
s=0.5

#generate true data
x_true,y_true = actual_distribution()

#fit on the actual data
f=numpy_poly_fitting(x_true,y_true,M=3)
#print('f:',f)
x_curve=np.linspace(x_true[0],x_true[-1],50)
y_curve=f(x_curve)

#add noise on the true data
y_noise=add_noise(y_curve)
x_noise=x_curve

#estimate the curve from the noisy data
x_noise_tmp = []
y_noise_tmp = []
tt = []
for i in range(0, len(x_noise)):
    t = np.random.randint(0,50)
    tt.append(t)

    x_noise_tmp.append(x_noise[t])
    y_noise_tmp.append(y_noise[t])

    imxx, mn = predictive_distribution(x_noise_tmp, y_noise_tmp, M, alpha, beta, s)
    S_star, M_star =  predictive_distribution_plot(x_noise, mn, imxx, s, beta)
    #print('x_noise_tmp =',x_noise_tmp) 
    
    y_est_list = []
    for i in range (0,len(M_star)):
        y_est_list.append(M_star[i][0])
        
    #show plot
    plt.plot(x_true,y_true,'ro')
    plt.plot(x_curve,y_curve,'red')
    plt.plot(x_noise,y_noise,'go')
    plt.plot(x_est,y_est)
    plt.errorbar(x_noise, y_est_list, yerr = S_star, fmt='b^', ecolor = 'blue',elinewidth=1, capsize = 3)
    #print('y_est_list =',y_est_list)
    print(tt)
    print(len(tt))
    plt.show()

start...
N = 1
M = 8


NameError: name 'w' is not defined